In [ ]:
# # !pip install -r https://raw.githubusercontent.com/Desire32/lora-ml-transfomers/main/requirements.txt --quiet

!pip install ipywidgets # huggin face widgets
!pip install --upgrade timm # timm error gpu gemma 3n
!pip install torchcodec
!pip install librosa soundfile

# audio errors
!sudo apt update
!sudo apt install -y ffmpeg
!pip install --upgrade huggingface_hub

# !pip install mlflow
# !pip install pyngrok

# # HF errors fix
!pip install datasets==3.6.0
# !pip index versions datasets
# !pip index versions numpy
# !pip install huggingface_hub==0.20.0

# !pip uninstall huggingface_hub transformers datasets
# !pip install huggingface_hub transformers datasets

In [ ]:
from huggingface_hub import login
login("")

In [ ]:
import logging
import torch
import warnings

logging.basicConfig(level=logging.INFO)
warnings.filterwarnings('ignore')
logging.getLogger("pyngrok").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("torch").setLevel(logging.ERROR)
logger = logging.getLogger(__name__)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
print(f"PyTorch version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

Using device: cuda
PyTorch version: 2.6.0+cu124
CUDA device: Tesla T4


In [ ]:
# !pip install --upgrade datasets
from transformers import AutoProcessor, AutoModelForCTC, Trainer, TrainingArguments, DataCollatorWithPadding
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import librosa
import numpy as np
import re

In [ ]:
from datasets import load_dataset

cv = load_dataset(
    "mozilla-foundation/common_voice_17_0",
    "en",
    split="train",
    streaming=True
)

# Взять первые 1000
cv_limited = list(cv.take(1000))
print(f"Загружено: {len(cv_limited)} примеров")

Размер: 563 примеров


In [ ]:
def audio_type_tester(dataset):
    sample = dataset['train'][0]
    audio_decoder = sample['audio']

    print("Attributes of AudioDecoder:")
    print([attr for attr in dir(audio_decoder) if not attr.startswith('_')])

    if hasattr(audio_decoder, 'path'):
        print(f"Path type: {type(audio_decoder.path)}")
        print(f"Path content: {audio_decoder.path}")

    for attr in ['file', 'filename', 'source', 'metadata']:
        if hasattr(audio_decoder, attr):
            value = getattr(audio_decoder, attr)
            print(f"{attr}: {type(value)} = {value}")

In [ ]:
audio_type_tester(ds)

KeyError: "Column train not in the dataset. Current columns in the dataset: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id']"

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForCTC, WhisperProcessor, WhisperForConditionalGeneration,Wav2Vec2Processor, Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
print(f"Модель: {type(model)}")
print(f"Процессор: {type(processor)}")
print(f"Параметры модели: {next(model.parameters()).dtype}")
print(f"Model vocab size: {model.config.vocab_size}")
print(f"Processor vocab: {len(processor.tokenizer.get_vocab())}")

Модель: <class 'transformers.models.wav2vec2.modeling_wav2vec2.Wav2Vec2ForCTC'>
Процессор: <class 'transformers.models.wav2vec2.processing_wav2vec2.Wav2Vec2Processor'>
Параметры модели: torch.float32
Model vocab size: 32
Processor vocab: 32


In [ ]:
sample = ds[0]
print(f"Sample keys: {sample.keys()}")
print(f"Audio type: {type(sample['audio'])}")
print(f"Sentence: {sample['transcription']}")

Sample keys: dict_keys(['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'])
Audio type: <class 'dict'>
Sentence: I would like to set up a joint account with my partner


In [ ]:
def sampling_map(array): # <- ds [train] goes here
    saved_array = array
    sr = array['audio']['sampling_rate']
    tr = 16000
    resample_array = librosa.resample(array['audio']['array'], orig_sr=sr, target_sr=tr)
    saved_array['audio'] = {
        'path': array['audio']['path'],
        'array': resample_array,
        'sampling_rate': tr
    }
    return saved_array

In [ ]:
from tqdm import tqdm
import copy

In [ ]:
# reforged_train = [sampling_map(sample) for sample in tqdm(ds['train'], desc="Resampling")]
# reforged_eval = [sampling_map(sample) for sample in tqdm(ds['validation'], desc="Resampling")]

small_train = [sampling_map(sample) for sample in tqdm(ds, desc="Resample")]

Resample: 100%|██████████| 563/563 [00:02<00:00, 209.98it/s]


In [ ]:
# reforged_train[0]
small_train[0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f9018fd3747971e77d59e6c5da3fdf9d5bb914c495e16c23e1fe47c921d76a7a/en-US~JOINT_ACCOUNT/602ba55abb1e6d0fbce92065.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/f9018fd3747971e77d59e6c5da3fdf9d5bb914c495e16c23e1fe47c921d76a7a/en-US~JOINT_ACCOUNT/602ba55abb1e6d0fbce92065.wav',
  'array': array([ 1.70562416e-05,  2.18727451e-04,  2.28099874e-04, ...,
          3.43842403e-05, -5.96364771e-06, -1.76846661e-05]),
  'sampling_rate': 16000},
 'transcription': 'I would like to set up a joint account with my partner',
 'english_transcription': 'I would like to set up a joint account with my partner',
 'intent_class': 11,
 'lang_id': 4}

In [ ]:
model = model.to('cpu')

In [ ]:
def process_reforged_list_wac2vec2(rf_ds):
    audio_arrays = [sample["audio"]["array"] for sample in rf_ds]
    # sentences = [sample["sentence"] for sample in rf_ds]
    sentences = [sample["transcription"] for sample in rf_ds]

    inputs = processor(
        audio_arrays,
        sampling_rate=16000,
        padding=True,
        max_length=16000,
        truncation=True
    )

    labels = processor.tokenizer(
        sentences,
        padding='max_length',
        max_length=512,
        truncation=True
    )

    # ПРАВИЛЬНО - работаем с input_ids, не с labels напрямую!
    labels_ids = labels["input_ids"]
    labels_ids = torch.tensor(labels_ids)  # Превращаем в tensor
    labels_ids[labels_ids == 0] = -100     # ВОТ ТАК РАБОТАЕТ!

    return {
        **inputs,
        "labels": labels_ids
    }
# def process_whisper_data(sample_list): -> крашится при работе с mozilla common voice, требует отрезки по 30 секунд а не по 5
#     audio_arrays = [sample["audio"]["array"] for sample in sample_list]
#     sentences = [sample["sentence"] for sample in sample_list]

#     # Whisper processor делает mel-спектрограммы
#     inputs = processor(
#         audio_arrays,
#         sampling_rate=16000,
#         return_tensors="pt",
#         padding="max_length",        # Важно!
#         max_length=480000,          # 30 секунд * 16000 Hz
#         truncation=True
#     )

#     # Whisper tokenizer
#     labels = processor.tokenizer(
#         sentences,
#         padding=True,
#         truncation=True,
#         return_tensors="pt"
#     )

#     return {
#         **inputs,
#         "labels": labels["input_ids"]
#     }

In [ ]:
# processed_data_train = process_reforged_list_wac2vec2(reforged_train)
# processed_data_eval = process_reforged_list_wac2vec2(reforged_eval)

small_processed = process_reforged_list_wac2vec2(small_train)

In [ ]:
small_processed['labels'][0]

tensor([  10,    4,    3,    3,    3,    3,    3,    4,    3,    3,    3,    3,
           4,    3,    3,    4,    3,    3,    3,    4,    3,    3,    4,    3,
           4,    3,    3,    3,    3,    3,    4,    3,    3,    3,    3,    3,
           3,    3,    4,    3,    3,    3,    3,    4,    3,    3,    4,    3,
           3,    3,    3,    3,    3,    3, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -1

In [ ]:
print(processed_data_train.keys())
print(processed_data_eval.keys())

dict_keys(['input_values', 'labels'])
dict_keys(['input_values', 'labels'])


In [ ]:
from datasets import Dataset
# train_hf = Dataset.from_dict(processed_data_train)
# eval_hf = Dataset.from_dict(processed_data_eval)

small_hf = Dataset.from_dict(small_processed)

In [ ]:
# print("=== TRAIN DATASET ===")
# print(f"Размер: {len(train_hf)}")
# print(f"Колонки: {train_hf.column_names}")
# print(f"Features: {train_hf.features}")

# print("\n=== EVAL DATASET ===")
# print(f"Размер: {len(eval_hf)}")
# print(f"Колонки: {eval_hf.column_names}")
# print(f"Features: {eval_hf.features}")

print("\n=== SMALL DATASET ===")
print(f"Размер: {len(small_hf)}")
print(f"Колонки: {small_hf.column_names}")
print(f"Features: {small_hf.features}")


=== SMALL DATASET ===
Размер: 563
Колонки: ['input_values', 'labels']
Features: {'input_values': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
# Проверить labels
print("Проверка labels:")
sample_labels = small_hf[0]['labels']
print(f"Labels: {sample_labels}")
print(f"Уникальные: {set(sample_labels)}")
print(f"Есть ли -100: {-100 in sample_labels}")
print(f"Все ли -100: {all(x == -100 for x in sample_labels)}")

Проверка labels:
Labels: [10, 4, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -

In [ ]:
# Проверить размеры
print(f"Model vocab size: {model.config.vocab_size}")
print(f"Tokenizer vocab size: {processor.tokenizer.vocab_size}")
print(f"Max label value: {max([max(x) for x in small_hf['labels'] if len(x) > 0])}")

Model vocab size: 32
Tokenizer vocab size: 32
Max label value: 29


In [ ]:
print(f"Pad token ID: {processor.tokenizer.pad_token_id}")

Pad token ID: 0


In [ ]:
training_args = TrainingArguments(
   output_dir="./test",
   per_device_train_batch_size=8,
   per_device_eval_batch_size=8,
   eval_strategy="steps",
   eval_steps=10,                 # Валидация каждые 10 шагов
   save_steps=10,                 # Сохранение каждые 10 шагов
   logging_steps=10,              # Логи каждые 10 шагов
   max_steps=100,                 # Ровно 100 шагов
   learning_rate=1e-6,
   weight_decay=0.005,
   warmup_steps=20,               # 20% от 100 шагов
   save_total_limit=2,
   load_best_model_at_end=True,   # Загрузить лучшую модель в конце
   metric_for_best_model="eval_loss",
   greater_is_better=False,       # Меньший loss = лучше
)

trainer = Trainer(
   model=model_2,
   args=training_args,
   train_dataset=train_hf,        # Весь датасет
   eval_dataset=eval_hf,
   data_collator=data_collator,
   tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: elormiden (elormiden-university-of-central-lancashire) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.000000
100,0.000000


KeyboardInterrupt: 

In [ ]:
# from transformers import TrainingArguments, Trainer
# training_args = TrainingArguments(
#     output_dir='/kaggle/working/openai-whisper-finetuned_mozilla',
#     run_name="wav2vec2-greek-asr",  # ← Добавили уникальное имя
#     overwrite_output_dir=True,
#     max_steps=500,
#     per_device_train_batch_size=4,
#     save_steps=50,
#     save_total_limit=1,
#     prediction_loss_only=True,
#     fp16=True,
#     learning_rate=5e-6,
#     logging_steps=10,
#     dataloader_pin_memory=True,
#     # eval_strategy="steps",
#     # eval_steps=50,
#     disable_tqdm=False,
#     report_to=[],
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_hf,
#     # eval_dataset=eval_hf,
#     tokenizer=processor.feature_extractor,
# )